In [ ]:
import os
import sys
from PIL import Image
from tqdm.notebook import tqdm
import pandas as pd

from transformers import pipeline

sys.path.append("../")

from const import *

In [ ]:
ethnicity_pipe = pipeline("image-classification", model="cledoux42/Ethnicity_Test_v003")
gender_pipe = pipeline("image-classification", model="rizvandwiki/gender-classification-2")
age_pipe = pipeline("image-classification", model="nateraw/vit-age-classifier")

In [ ]:
faces_info = FACES_COLLECTION.get()
faces_info.keys()

In [ ]:
stats = []
for metadata in tqdm(faces_info["metadatas"]):
    filename = os.path.join(PRADO_PATH, "images", metadata["image_id"])
    im = Image.open(filename)
    
    top = metadata['fl_top']
    right = metadata['fl_right']
    bottom = metadata['fl_bottom']
    left = metadata['fl_left']
    
    face = im.crop((left, top, right, bottom))
    
    ethnicity = ethnicity_pipe(face)[0]['label']
    gender = gender_pipe(face)[0]['label']
    age = age_pipe(face)[0]['label']
    
    stats.append({
        "ethnicity": ethnicity,
        "gender": gender,
        "age": age,
    })

In [ ]:
stats_df = pd.DataFrame(stats)
stats_df.head()

In [ ]:
stats_df.to_csv("stats.csv", index=False)

In [ ]:
pd.read_csv("stats.csv")